## Required Libs

In [ ]:
pip install SpeechRecognition

In [ ]:
pip install textblob

In [ ]:
pip install vosk


In [ ]:
pip install pyspellchecker


In [ ]:
#vosk model languange download
#https://alphacephei.com/vosk/models
#portuguese: https://alphacephei.com/vosk/models/vosk-model-pt-fb-v0.1.1-20220516_2113.zip
#portuguese small: https://alphacephei.com/vosk/models/vosk-model-small-pt-0.3.zip


## Only Transcript

In [ ]:
import speech_recognition as sr


r = sr.Recognizer()
with sr.AudioFile(r'C:\Users\PC\Downloads\file.wav') as source:
    audio_data = r.record(source)
    
    try:
        text = r.recognize_google(audio_data, language='pt-BR')
        print("Transcrição: ", text)
    except sr.UnknownValueError:
        print("Google Speech Recognition não conseguiu entender o áudio")
    except sr.RequestError as e:
        print(f"Não foi possível solicitar resultados do serviço Google Speech Recognition; {e}")


## Transcripted default with spellcheck

In [ ]:
import speech_recognition as sr
from spellchecker import SpellChecker

# Função para corrigir texto
def corrigir_texto(texto):
    spell = SpellChecker(language='pt')
    palavras = texto.split()
    palavras_corrigidas = [spell.correction(palavra) for palavra in palavras]
    texto_corrigido = ' '.join(palavras_corrigidas)
    return texto_corrigido

# Instancia o reconhecedor
r = sr.Recognizer()

# Carrega o arquivo de áudio
with sr.AudioFile(r'C:\Users\PC\Downloads\file.wav') as source:
    audio_data = r.record(source)

    # Tenta transcrever o áudio para texto
    try:
        texto_transcrito = r.recognize_google(audio_data, language='pt-BR')
        print("Transcrição Original: ", texto_transcrito)
        
        # Corrige o texto transcrito
        texto_corrigido = corrigir_texto(texto_transcrito)
        print("Transcrição Corrigida: ", texto_corrigido)
        
    except sr.UnknownValueError:
        print("Google Speech Recognition não conseguiu entender o áudio")
    except sr.RequestError as e:
        print(f"Não foi possível solicitar resultados do serviço Google Speech Recognition; {e}")


## Transcribe and correct the text method 1 with textblob

In [ ]:
import speech_recognition as sr
from textblob import TextBlob

def corrigir_texto(texto):
    texto_corrigido = TextBlob(texto)
    return texto_corrigido.correct()

def transcrever_e_corrigir(caminho_do_arquivo):
    r = sr.Recognizer()
    with sr.AudioFile(caminho_do_arquivo) as source:
        audio_data = r.record(source)
        
        try:
            texto_transcrito = r.recognize_google(audio_data, language='pt-BR')
            print("Transcrição Original: ", texto_transcrito)
            
            texto_corrigido = corrigir_texto(texto_transcrito)
            print("Transcrição Corrigida: ", texto_corrigido)
            
        except sr.UnknownValueError:
            print("Google Speech Recognition não conseguiu entender o áudio")
        except sr.RequestError as e:
            print(f"Não foi possível solicitar resultados do serviço Google Speech Recognition; {e}")

# Substitua 'caminho_para_seu_arquivo_de_audio.wav' pelo caminho real do seu arquivo de áudio
caminho_do_arquivo = r'C:\Users\PC\Downloads\file.wav'
transcrever_e_corrigir(caminho_do_arquivo)


## Transcribe and correct the text method 2 with model vosk textblob

In [ ]:
from vosk import Model, KaldiRecognizer
import wave
import json
import os

def transcrever_vosk(caminho_do_arquivo, caminho_do_modelo='modelo_vosk/'):
    if not os.path.exists(caminho_do_modelo):
        print("Modelo Vosk não encontrado em:", caminho_do_modelo)
        return

    # Carregar o modelo Vosk
    model = Model(caminho_do_modelo)
    rec = KaldiRecognizer(model, 16000)

    # Abrir o arquivo de áudio com wave
    try:
        with wave.open(caminho_do_arquivo, "rb") as wf:
            # Configurar o reconhecedor com a taxa de amostragem do arquivo
            rec = KaldiRecognizer(model, wf.getframerate())
            rec.SetWords(True)  # Para obter a saída com palavras reconhecidas
            texto_transcrito = ""

            # Processar o arquivo de áudio e obter a transcrição
            while True:
                data = wf.readframes(4000)
                if len(data) == 0:
                    break
                if rec.AcceptWaveform(data):
                    partial_result = json.loads(rec.Result())
                    texto_transcrito += partial_result.get('text', '') + " "
            
            # Obter resultados finais
            final_result = json.loads(rec.FinalResult())
            texto_transcrito += final_result.get('text', '')

            print("Transcrição Original: ", texto_transcrito)
    except wave.Error as e:
        print("Erro ao abrir o arquivo de áudio:", e)
    except Exception as e:
        print("Erro durante a transcrição:", e)

# Substitua os caminhos abaixo conforme necessário
caminho_do_arquivo = r'C:\Users\PC\Downloads\file.wav'
caminho_do_modelo = r'C:\Users\PC\Documents\vosk-model-small-pt-0.3'
transcrever_vosk(caminho_do_arquivo, caminho_do_modelo)
